In [1]:
import pandas as pd
import string
import unicodedata
import os

In [2]:
def unicodeToAscii(s):
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
            and c in ALL_LET
        )
ALL_LET = string.ascii_letters + " .,;'"
N_HIDDEN = 128
N_LET = len(ALL_LET)

def clean_name(name, f_name = False):
    if f_name:
        name = name.split(" ")[0]
    name = name.lower()
    name = unicodeToAscii(name)
    return name

## Cleaning Parlimentary Elections Elections

In [3]:
def clean_milletvekili(path):
    city = (path.split("_")[-1]).split(".")[0]
    df = pd.read_excel(f"Data/{path}", skiprows = lambda x: x in [i for i in range(0, 9)])
    df.rename(columns = {
                       "Unnamed: 0": "Variable",
                       "Unnamed: 1": "Type"
                        }, inplace = True)
    df = df[df.Type != "C"]
    df.Type = df.Type.str.replace("A", "Total Votes")
    df.Type = df.Type.str.replace("B", "Percent Votes")
    df = df.T
    df.columns = df.loc["Variable"]
    df = df.drop("Variable")
    columns = df.columns
    types = list(df.loc["Type"])
    types = ["" if str(i) == "nan" else i for i in types]
    columns = [str(columns[i]) + "_" + str(types[i]) for i in range(0, len(columns))]
    columns = [clean_name(i.replace("ı", "i")) for i in columns]
    columns = [i.replace("  ", "") for i in columns]
    columns = [i.replace(" ", "_") for i in columns]
    replace = ["registered_voter", "turnout", "percent_turnout", "valid_vote", "num_milletvekili"]
    for i in range(0, len(replace)):
        columns[i] = replace[i]
    df.columns = columns
    df = df.drop("Type")
    df["city"] = [city] * len(df)
    df = df.reset_index()
    df.rename(columns = {'index':'year'}, inplace = True)
    if 'nan' in df.columns:
        df = df.drop(columns = ['nan'])
    return df

In [4]:
milletvekili_raw = [i for i in os.listdir("Data/") if "milletvekili_" in i ]
milletvekili_cleaned = [clean_milletvekili(i) for i in milletvekili_raw]

In [5]:
df_final = pd.concat(milletvekili_cleaned, ignore_index = True)
df_final.year = df_final.year.replace('1 Kasım 2015', '2015_1')
df_final.year = df_final.year.replace('7 Haziran 2015', '2015_2')
df_final = df_final[df_final.year != 'Unnamed: 13']
df_final.to_csv('clean_data/turkish_parlimentary.csv', index = False)

## Cleaning 2018 & 2015 Detailed Parlimentary Election Data

In [6]:
def clean_detailed_milletvekili(path):
    df = pd.read_excel(f'Data/{path}', skiprows = lambda x: x in [i for i in range(0, 7)])
    year = path.split('_')[0]
    if year == '2018':
        columns = ['election_city', 'election_source', 'registered_voters', 'turnout', 'percent_turnout',
                     'valid_vote', 'AKP', 'MHP', 'huda', 'vatan', 'HDP', 'CHP', 'saadet', 'iyi', 'independent']
        columns = [f'2018_{i}' for i in columns[1:]]
    elif 'kasim' in year:

        columns = ['election_city', 'election_source', 'registered_voters', 'turnout', 'percent_turnout',
                     'valid_vote', 'millet', 'vatan', 'CHP', 'hak', 'saadet', 'dsp', 'dp', 'btp',
                     'MHP', 'hkp', 'ldp', 'HDP', 'bbp', 'AKP', 'kp', 'dyp', 'indep']
        columns = [f'2015-1_{i}' for i in columns[1:]]
    else:
        columns = ['election_city', 'election_source', 'registered_voters', 'turnout', 'percent_turnout',
                     'valid_vote', 'dyp', 'anadolu', 'hak', 'kp', 'millet', 'hap', 'mep', 'turk', 'hkp', 'ldp',
                     'MHP', 'HDP', 'saadet', 'CHP', 'AKP', 'DSP', 'yurt', 'DP', 'vatan', 'BTP', 'indep']
        columns = [f'2015-2_{i}' for i in columns[1:]]
    df.columns = ['election_city'] + columns
    df = df[(df[df.columns[1]] == 'Ülke geneli') | (df[df.columns[1]] == 'Seçim çevresi')]
    df.drop(columns = [df.columns[1]], inplace = True)
    return df

In [7]:
paths = ['2018_milletvekili.xlsx', '2015-1kasim_milletvekili.xlsx', '2015-7haziran_milletvekili.xlsx']
milletvekili_detailed_cleaned = [clean_detailed_milletvekili(i) for i in paths]

In [8]:
df_final = milletvekili_detailed_cleaned[0].merge(milletvekili_detailed_cleaned[1], on = 'election_city', how = 'left')
df_final = df_final.merge(milletvekili_detailed_cleaned[2], on = 'election_city', how = 'left')
df_final['election_city'] = df_final['election_city'].apply(lambda row: clean_name(row))
df_final.to_csv('clean_data/milletvekili_detailed.csv', index = 'False')

## Cleaning Municipality Election Data

In [9]:
df = pd.read_excel("Data/1989_BB.xlsx", skiprows = lambda x: x in [i for i in range(0, 7)])

In [10]:
def fill_col_names(cols, year):
    cols_final = ["il"]
    for col in cols[1:]:
        index = cols.index(col)
        if "Unnamed" in col:
            cols_final.append(clean_name(cols[index - 1].replace("ı", "i")) + "_" + "percent" + "_" + year)
            cols_final[index - 1] = cols[index - 1] + "_" + "vote" + "_" + year
        else:
            if col != "il":
                cols_final.append(clean_name(col.replace("ı", "i")) + "_"+ year)
    return cols_final

In [11]:
def clean_municipality_elec(path):
    df = pd.read_excel(f'Data/{path}', skiprows = lambda x: x in [i for i in range(0, 7)])
    year = path.split("_")[0]
    df.columns = fill_col_names(list(df.columns), year)
    df = df.drop(0)
    if f"belediye_{year}" in df.columns:
        df = df[df[f"belediye_{year}"].isna()]
        df.drop(columns = [f"belediye_{year}", f"sandik sayisi_{year}"], inplace = True)
    else:
        df = df[df[f"ilce_{year}"].isna()]
        df.drop(columns = [f"ilce_{year}", f"baskanlik sayisi_{year}"], inplace = True)
    df = df[df.il.notna()]
    df.il = df.il.apply(lambda row: clean_name(row))
    df.il = df.il.str.replace(" ", "")
    df.rename(columns = {
    "kayitli secmen sayisi": 'registered_voters', 
    "oy kullanan secmen sayisi": 'turnout', 
    "secime katilma orani ": 'percent_turnout',
    "gecerli oy sayisi": "valid_vote"
                        }, inplace = True)
    return df

In [12]:
municipality_paths = [i for i in os.listdir("Data/") if "BB" in i ]
municipality_paths_clean = [clean_municipality_elec(i) for i in municipality_paths]

In [13]:
df_final = municipality_paths_clean[-7]
for i in range(-6, 0, 1):
    df_final = df_final.merge(municipality_paths_clean[i], on = "il", how = "left")
df_final.to_csv("clean_data/turkey_municipality.csv", index = "False")